In [1]:
import torchio as tio
from torchio.data.io import sitk_to_nib
import SimpleITK as sitk
import napari
import numpy as np
import nibabel as nib

from utils.class_SAMMed3D import SAMMed3DInferer
#import utils.class_SAMMed3D as c
from utils.prompt_3d import get_pos_clicks3D
import utils.analysis as anUt
from utils.interactivity import iterate_3d
from utils.image import read_im_gt, read_reorient_nifti

from utils.image import  read_reorient_nifti

# Obtain model, image, gt
device = 'cuda'
checkpoint_path = '/home/t722s/Desktop/UniversalModels/TrainedModels/sam_med3d_turbo.pth'

inferer = SAMMed3DInferer(checkpoint_path, device)

img_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/imagesTr/BDMAP_00000001_0000.nii.gz'
gt_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz'
class_label = 3

gt = nib.load(gt_path).get_fdata()
gt = np.where(gt == class_label, 1, 0)

# Set image to predict on 
inferer.set_image(img_path)

In [3]:
# Experiment: 5 points per volume
seed = 11121
n = 5
pts_prompt = get_pos_clicks3D(gt, n, seed = seed)

#pred = inferer.predict(img, pts_prompt, cheat = True, gt = gt)
segmentation = inferer.predict(pts_prompt, store_patching=True).get_fdata()

anUt.compute_dice(segmentation, gt)

0.8319491632062413

In [4]:
# Experiment: iterate on points
pass_prev_prompts = SAMMed3DInferer.pass_prev_prompts
seed = 3
perf_bound = 0.9
dof_bound = 18


dice_scores, dofs = iterate_3d(inferer, gt, SAMMed3DInferer.pass_prev_prompts, perf_bound, dof_bound, seed, sammed3d=True)
dice_scores

[0.0559962178914745,
 0.14349487876568723,
 0.5251296558227251,
 0.7738505044424797,
 0.7985250705090922,
 0.8098176204306984,
 0.8204396549543397,
 0.8409741754679707,
 0.8708920062821093,
 0.8871685978141528,
 0.8847824699993619,
 0.85995838872561]